In [2]:
import numpy as np   # 수 연산
import pandas as pd   # 데이터분석
import matplotlib.pyplot as plt    # 시각화
import seaborn as sns   # 샘플 + 화려함

import matplotlib.font_manager as fm # 한글폰트를 쓰기 위한 방법(?)
font_name = fm.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc("font", family=font_name)

import matplotlib as mpl
mpl.rcParams["axes.unicode_minus"] = False

import operator
from numpy import linalg

#머신러닝 관련(사이킷런)
import mglearn

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, GridSearchCV



In [1]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve
from sklearn.metrics import precision_score, recall_score

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler

from sklearn.linear_model import LinearRegression

from sklearn.svm import LinearSVR

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from xgboost import plot_importance, plot_tree

from lightgbm import LGBMClassifier, LGBMRegressor
from lightgbm import plot_importance, plot_tree, plot_metric

from sklearn.svm import SVR, SVC

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

import statsmodels.api as sm

# 구내식당 중식 인원 머신러닝
+ 일반 선형회귀, 릿지모델, 라쏘모델
+ XGBRegressor
+ LGBMResgressor
+ SVR

In [96]:
ln_train = pd.read_csv("../byun/data/lunch_df_encoding.csv", encoding="utf-8")
ln_train

,datetime,year,month,date,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,...,season_winter,weekdays_friday,weekdays_monday,weekdays_thursday,weekdays_tuesday,weekdays_wednesday,vacation_N,vacation_Y,new_lunch_N,new_lunch_Y
0,2016-02-01,2016,2,1,2601,2401,50,150,238,0,...,1,0,1,0,0,0,1,0,1,0
1,2016-02-02,2016,2,2,2601,2378,50,173,319,0,...,1,0,0,0,1,0,1,0,1,0
2,2016-02-03,2016,2,3,2601,2365,56,180,111,0,...,1,0,0,0,0,1,1,0,1,0
3,2016-02-04,2016,2,4,2601,2277,104,220,355,0,...,1,0,0,1,0,0,1,0,1,0
4,2016-02-05,2016,2,5,2601,2142,278,181,34,0,...,1,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,2021,1,20,2983,2319,75,198,4,391,...,1,0,0,0,0,1,1,0,1,0
1201,2021-01-21,2021,1,21,2983,2309,92,231,462,351,...,1,0,0,1,0,0,1,0,1,0
1202,2021-01-22,2021,1,22,2983,2177,255,248,1,303,...,1,1,0,0,0,0,1,0,1,0
1203,2021-01-25,2021,1,25,2983,2396,107,153,616,327,...,1,0,1,0,0,0,1,0,1,0


In [97]:
ln_train.info()
ln_train.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   datetime               1205 non-null   object 
 1   year                   1205 non-null   int64  
 2   month                  1205 non-null   int64  
 3   date                   1205 non-null   int64  
 4   worker_number          1205 non-null   int64  
 5   real_number            1205 non-null   int64  
 6   vacation_number        1205 non-null   int64  
 7   biztrip_number         1205 non-null   int64  
 8   overtime_number        1205 non-null   int64  
 9   telecom_number         1205 non-null   int64  
 10  temperature            1204 non-null   float64
 11  rain                   1205 non-null   float64
 12  wind                   1204 non-null   float64
 13  humidity               1205 non-null   float64
 14  discomfort_index       1204 non-null   float64
 15  perc

(1205, 31)

In [98]:
ln_train.isnull().sum() #칼럼별 결측치 수

datetime                 0
year                     0
month                    0
date                     0
worker_number            0
real_number              0
vacation_number          0
biztrip_number           0
overtime_number          0
telecom_number           0
temperature              1
rain                     0
wind                     1
humidity                 0
discomfort_index         1
perceived_temperature    2
lunch_rice               0
lunch_number             0
season_fall              0
season_spring            0
season_summer            0
season_winter            0
weekdays_friday          0
weekdays_monday          0
weekdays_thursday        0
weekdays_tuesday         0
weekdays_wednesday       0
vacation_N               0
vacation_Y               0
new_lunch_N              0
new_lunch_Y              0
dtype: int64

In [99]:
ln_train = ln_train.dropna()
ln_train

,datetime,year,month,date,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,...,season_winter,weekdays_friday,weekdays_monday,weekdays_thursday,weekdays_tuesday,weekdays_wednesday,vacation_N,vacation_Y,new_lunch_N,new_lunch_Y
0,2016-02-01,2016,2,1,2601,2401,50,150,238,0,...,1,0,1,0,0,0,1,0,1,0
1,2016-02-02,2016,2,2,2601,2378,50,173,319,0,...,1,0,0,0,1,0,1,0,1,0
2,2016-02-03,2016,2,3,2601,2365,56,180,111,0,...,1,0,0,0,0,1,1,0,1,0
3,2016-02-04,2016,2,4,2601,2277,104,220,355,0,...,1,0,0,1,0,0,1,0,1,0
4,2016-02-05,2016,2,5,2601,2142,278,181,34,0,...,1,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,2021,1,20,2983,2319,75,198,4,391,...,1,0,0,0,0,1,1,0,1,0
1201,2021-01-21,2021,1,21,2983,2309,92,231,462,351,...,1,0,0,1,0,0,1,0,1,0
1202,2021-01-22,2021,1,22,2983,2177,255,248,1,303,...,1,1,0,0,0,0,1,0,1,0
1203,2021-01-25,2021,1,25,2983,2396,107,153,616,327,...,1,0,1,0,0,0,1,0,1,0


In [108]:
ln_test = pd.read_csv("../byun/data/lunch_test_encoding.csv", encoding="utf-8")
ln_test

,datetime,year,month,date,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,...,season_winter,weekdays_friday,weekdays_monday,weekdays_thursday,weekdays_tuesday,weekdays_wednesday,vacation_N,vacation_Y,new_lunch_N,new_lunch_Y
0,2021-01-27,2021,1,27,2983,2355.0,88,182,5,358.0,...,1,0,0,0,0,1,1,0,1,0
1,2021-01-28,2021,1,28,2983,2319.0,104,212,409,348.0,...,1,0,0,1,0,0,1,0,1,0
2,2021-01-29,2021,1,29,2983,2170.0,270,249,0,294.0,...,1,1,0,0,0,0,1,0,1,0
3,2021-02-01,2021,2,1,2924,2340.0,108,154,538,322.0,...,1,0,1,0,0,0,1,0,1,0
4,2021-02-02,2021,2,2,2924,2362.0,62,186,455,314.0,...,1,0,0,0,1,0,1,0,1,0
5,2021-02-03,2021,2,3,2924,2380.0,59,199,5,286.0,...,1,0,0,0,0,1,1,0,1,0
6,2021-02-04,2021,2,4,2924,2364.0,61,211,476,288.0,...,1,0,0,1,0,0,1,0,1,0
7,2021-02-05,2021,2,5,2924,2247.0,169,252,0,256.0,...,1,1,0,0,0,0,1,0,1,0
8,2021-02-08,2021,2,8,2924,2333.0,88,174,690,329.0,...,1,0,1,0,0,0,1,0,1,0
9,2021-02-09,2021,2,9,2924,2318.0,94,183,542,329.0,...,1,0,0,0,1,0,1,0,1,0


## 선형회귀(일반)

In [101]:
X_train = ln_train[['year','month','date','worker_number','real_number','vacation_number','biztrip_number','overtime_number','telecom_number',
             'temperature','rain','wind','humidity','discomfort_index','perceived_temperature','lunch_rice','season_fall','season_spring',
             'season_summer','season_winter','weekdays_friday','weekdays_monday','weekdays_thursday','weekdays_tuesday','weekdays_wednesday',
             'vacation_N','vacation_Y','new_lunch_N','new_lunch_Y']]

y_train = ln_train["lunch_number"]

In [102]:
lr = LinearRegression().fit(X_train, y_train)

In [103]:
print("훈련 점수 : ", lr.score(X_train, y_train))

훈련 점수 :  0.7743430539169095


In [104]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=0)
lr = LinearRegression().fit(X_train, y_train)

In [105]:
print("훈련 점수 : ", lr.score(X_train, y_train))
print("테스트 점수 : ", lr.score(X_test, y_test))

훈련 점수 :  0.7848986173519543
테스트 점수 :  0.7964245248497636


## 릿지회귀모델

In [106]:
from sklearn.linear_model import Ridge

ridge = Ridge().fit(X_train, y_train) # Ridge(alpha=1.0) 이 디폴트

print("훈련 점수 : ", ridge.score(X_train, y_train))
print("테스트 점수 : ", ridge.score(X_test, y_test))

ridge10 = Ridge(alpha=10).fit(X_train, y_train) # Ridge(alpha=1.0) 이 디폴트

print("훈련 점수 : ", ridge10.score(X_train, y_train))
print("테스트 점수 : ", ridge10.score(X_test, y_test))


ridge01 = Ridge(alpha=0.000000001).fit(X_train, y_train) # Ridge(alpha=1.0) 이 디폴트

print("훈련 점수 : ", ridge01.score(X_train, y_train))
print("테스트 점수 : ", ridge01.score(X_test, y_test))

훈련 점수 :  0.785750892409135
테스트 점수 :  0.7939237427269235
훈련 점수 :  0.7847089874999071
테스트 점수 :  0.7936329969012887
훈련 점수 :  0.785764424924516
테스트 점수 :  0.7938239417025961


## 라쏘회귀모델

In [107]:
from sklearn.linear_model import Lasso

lasso = Lasso().fit(X_train, y_train) #Lasso(alpha=1) 디폴트

print("훈련 점수 : ", lasso.score(X_train, y_train))
print("테스트 점수 : ", lasso.score(X_test, y_test)) 

print("사용한 특성의 수 : ", np.sum(lasso.coef_ != 0)) 
lasso001 = Lasso(alpha=0.01).fit(X_train, y_train)

print("훈련 점수 : ", lasso001.score(X_train, y_train))
print("테스트 점수 : ", lasso001.score(X_test, y_test))

print("사용한 특성의 수 : ", np.sum(lasso001.coef_ != 0))

lasso00001 = Lasso(alpha=0.000000001).fit(X_train, y_train)

print("훈련 점수 : ", lasso00001.score(X_train, y_train))
print("테스트 점수 : ", lasso00001.score(X_test, y_test)) 

print("사용한 특성의 수 : ", np.sum(lasso00001.coef_ != 0))

훈련 점수 :  0.7817039925408038
테스트 점수 :  0.7883245851474749
사용한 특성의 수 :  24
훈련 점수 :  0.7857636760204345
테스트 점수 :  0.7937909609765834
사용한 특성의 수 :  28
훈련 점수 :  0.7857643487990997
테스트 점수 :  0.7938014642621193
사용한 특성의 수 :  27


C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 59495.01217186265, tolerance: 3931.371726385809
  model = cd_fast.enet_coordinate_descent(
C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 239452.64568418264, tolerance: 3931.371726385809
  model = cd_fast.enet_coordinate_descent(
C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4211199.470651516, tolerance: 3931.371726385809
  model = cd_fast.enet_coordinate_descent(


# 구내식당 석식 인원 예측 모델

In [109]:
dn_train = pd.read_csv("../byun/data/dinner_df_encoding.csv", encoding="utf-8")
dn_train

,datetime,year,month,date,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,...,season_winter,weekdays_friday,weekdays_monday,weekdays_thursday,weekdays_tuesday,weekdays_wednesday,vacation_N,vacation_Y,new_dinner_N,new_dinner_Y
0,2016-02-01,2016,2,1,2601,2401,50,150,238,0,...,1,0,1,0,0,0,1,0,1,0
1,2016-02-02,2016,2,2,2601,2378,50,173,319,0,...,1,0,0,0,1,0,1,0,1,0
2,2016-02-03,2016,2,3,2601,2365,56,180,111,0,...,1,0,0,0,0,1,1,0,1,0
3,2016-02-04,2016,2,4,2601,2277,104,220,355,0,...,1,0,0,1,0,0,1,0,1,0
4,2016-02-05,2016,2,5,2601,2142,278,181,34,0,...,1,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,2021,1,20,2983,2319,75,198,4,391,...,1,0,0,0,0,1,1,0,1,0
1201,2021-01-21,2021,1,21,2983,2309,92,231,462,351,...,1,0,0,1,0,0,1,0,1,0
1202,2021-01-22,2021,1,22,2983,2177,255,248,1,303,...,1,1,0,0,0,0,1,0,1,0
1203,2021-01-25,2021,1,25,2983,2396,107,153,616,327,...,1,0,1,0,0,0,1,0,1,0


In [110]:
dn_test = pd.read_csv("../byun/data/dinner_test_encoding.csv", encoding="utf-8")
dn_test

,datetime,year,month,date,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,...,season_winter,weekdays_friday,weekdays_monday,weekdays_thursday,weekdays_tuesday,weekdays_wednesday,vacation_N,vacation_Y,new_dinner_N,new_dinner_Y
0,2021-01-27,2021,1,27,2983,2355.0,88,182,5,358.0,...,1,0,0,0,0,1,1,0,0,1
1,2021-01-28,2021,1,28,2983,2319.0,104,212,409,348.0,...,1,0,0,1,0,0,1,0,1,0
2,2021-01-29,2021,1,29,2983,2170.0,270,249,0,294.0,...,1,1,0,0,0,0,1,0,1,0
3,2021-02-01,2021,2,1,2924,2340.0,108,154,538,322.0,...,1,0,1,0,0,0,1,0,1,0
4,2021-02-02,2021,2,2,2924,2362.0,62,186,455,314.0,...,1,0,0,0,1,0,1,0,1,0
5,2021-02-03,2021,2,3,2924,2380.0,59,199,5,286.0,...,1,0,0,0,0,1,1,0,1,0
6,2021-02-04,2021,2,4,2924,2364.0,61,211,476,288.0,...,1,0,0,1,0,0,1,0,1,0
7,2021-02-05,2021,2,5,2924,2247.0,169,252,0,256.0,...,1,1,0,0,0,0,1,0,1,0
8,2021-02-08,2021,2,8,2924,2333.0,88,174,690,329.0,...,1,0,1,0,0,0,1,0,1,0
9,2021-02-09,2021,2,9,2924,2318.0,94,183,542,329.0,...,1,0,0,0,1,0,1,0,1,0
